## <b>Weather forecast project 

# <b>05 - MODEL FOR IMBALANCED DATA    

### Packages nécessaires au notebook :

In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import bokeh
import plotly
import sys
import geopy.distance
from collections import defaultdict
import geopandas as gpd
from shapely.geometry import Point
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.neighbors import DistanceMetric
from math import radians
from tqdm import tqdm
tqdm.pandas()
import pickle
import time
import lazypredict
from lazypredict.Supervised import LazyClassifier

from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score, ConfusionMatrixDisplay, precision_score, recall_score, f1_score, classification_report, roc_curve, auc, precision_recall_curve, average_precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, validation_curve
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression,f_classif,chi2

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")

plt.rcParams['figure.figsize'] = [15, 10]

import plotly.express as px

from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer

from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler,  ClusterCentroids
from imblearn.metrics import classification_report_imbalanced

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler,  ClusterCentroids
from imblearn.metrics import classification_report_imbalanced, geometric_mean_score
from imblearn.ensemble import BalancedRandomForestClassifier


from sklearn.metrics import f1_score
from sklearn.svm import SVC


import xgboost as xgb

np.random.seed(0)

###### <b>Import des données

In [3]:
df = pd.read_csv("../data/data_features.csv").iloc[:,2:] #iloc en attendant de corriger le notebook3

###### <b>Séparation des données en variables explicatives et variable cible

In [4]:
X = df.drop(['raintomorrow'],axis=1)
y = df['raintomorrow']

In [15]:
y.value_counts(normalize=True)

0   0.84
1   0.16
Name: raintomorrow, dtype: float64

> Dans ce dataset, 16% des données correspondent à une prédiction de pluie le lendemain, ce qui signifie qu'un modèle naïf qui prédit qu'il ne pleut pas systématiquement, obtiendrait déjà un score de 84 % d'accuracy.

###### <b>Fractionnement des données en ensemble d'entraînement et de test

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 10)

###### <b>Sur-échantillonnage (Oversampling) aléatoire

In [16]:
rOs = RandomOverSampler()
X_ro, y_ro = rOs.fit_resample(X_train, y_train)
print('Classes échantillon oversampled :', dict(pd.Series(y_ro).value_counts()))

Classes échantillon oversampled : {1: 48841, 0: 48841}


###### <b>Sous-échantillonnage (Undersampling) aléatoire

In [17]:
rUs = RandomUnderSampler()
X_ru, y_ru = rUs.fit_resample(X_train, y_train)
print('Classes échantillon undersampled :', dict(pd.Series(y_ru).value_counts()))

Classes échantillon undersampled : {0: 9324, 1: 9324}


## 1er modèle : SVM simple

#### <b> SVC avec un sur-échantillon aléatoire

In [20]:
# Instanciation d'un objet SVM avec une fonction de noyau RBF et gamma='scale'
# La fonction de noyau RBF (Radial Basis Function) est une fonction qui peut transformer des données dans un espace de dimension supérieure pour faciliter leur séparation.
# gamma est un paramètre de la fonction de noyau RBF. Ici, gamma='scale' signifie que la valeur de gamma sera calculée automatiquement en fonction de la taille de l'échantillon d'entraînement.
svm = SVC(gamma='scale')

# Entrainement d'un modèle SVM sur les données d'entraînement
# La méthode fit() est utilisée pour entraîner le modèle SVM sur les données d'entraînement X_ro et y_ro.
svm.fit(X_ro, y_ro)

# Prédiction des étiquettes de classe pour les données de test
# La méthode predict() est utilisée pour prédire les étiquettes de classe pour les données de test X_test.
y_pred = svm.predict(X_test)

# Création d'une table de contingence (matrice de confusion) montrant le nombre de prédictions correctes et incorrectes pour chaque classe
# La fonction pd.crosstab() est utilisée pour créer une table de contingence (matrice de confusion) qui montre le nombre de prédictions correctes et incorrectes pour chaque classe.
# Les vraies étiquettes de classe y_test et les étiquettes de classe prédites y_pred sont utilisées comme arguments.
contingency_table = pd.crosstab(y_test, y_pred)
print(contingency_table)

# Affichage d'un rapport de classification qui fournit des métriques de performance pour chaque classe
# La fonction classification_report_imbalanced() est utilisée pour calculer les métriques de performance pour chaque classe.
# Cette fonction est spécifiquement conçue pour les ensembles de données déséquilibrés, où certaines classes ont beaucoup plus d'exemples que d'autres.
report = classification_report_imbalanced(y_test, y_pred)
print(report)

col_0            0     1
raintomorrow            
0             9570  2670
1              633  1669
                   pre       rec       spe        f1       geo       iba       sup

          0       0.94      0.78      0.73      0.85      0.75      0.57     12240
          1       0.38      0.73      0.78      0.50      0.75      0.56      2302

avg / total       0.85      0.77      0.73      0.80      0.75      0.57     14542



> * En analysant le rapport de classification, nous pouvons conclure que le modèle SVM entraîné sur les données d'entraînement déséquilibrées a une performance moyenne sur les données de test déséquilibrées.
> * La métrique de rappel (recall) est relativement élevée pour la classe minoritaire (classe 1), ce qui indique que le modèle est relativement bon à identifier les vrais positifs. 
> * Cependant, la précision (precision) est relativement faible pour la même classe, ce qui signifie que le modèle peut également produire un grand nombre de faux positifs. 
> * La métrique F1-score est une mesure harmonique entre la précision et le rappel et est donc une mesure plus équilibrée de la performance globale du modèle. 
> * Dans ce cas, le score F1 est d'environ 0,5 pour la classe 1 minoritaire. 
> * Cela indique que le modèle peut être amélioré pour mieux identifier les exemples de la classe minoritaire

#### <b> SVC avec un sous-échantillon aléatoire

In [18]:
svm = SVC(gamma='scale')
svm.fit(X_ru, y_ru)

y_pred = svm.predict(X_test)
print(pd.crosstab(y_test, y_pred))
print(classification_report_imbalanced(y_test, y_pred))

col_0            0     1
raintomorrow            
0             9130  3110
1              633  1669
                   pre       rec       spe        f1       geo       iba       sup

          0       0.94      0.75      0.73      0.83      0.74      0.54     12240
          1       0.35      0.73      0.75      0.47      0.74      0.54      2302

avg / total       0.84      0.74      0.73      0.77      0.74      0.54     14542



In [22]:
# Instanciation d'un objet SVC avec le paramètre gamma défini sur "scale"
svm = SVC(gamma='scale')

# Entraînement du modèle sur les données d'entraînement équilibrées (X_ru et y_ru) à l'aide de la méthode fit()
svm.fit(X_ru, y_ru)

# Prédiction des étiquettes de classe pour les données de test à l'aide de la méthode predict()
y_pred = svm.predict(X_test)

# Création de la matrice de confusion à partir des étiquettes de classe prédites et des étiquettes de classe réelles en utilisant la fonction crosstab() de pandas
confusion_matrix = pd.crosstab(y_test, y_pred)

# Affichage de la matrice de confusion à l'aide de la fonction print()
print(confusion_matrix)

# Calcul et affichage du rapport de classification équilibré à l'aide de la fonction classification_report_imbalanced()
print(classification_report_imbalanced(y_test, y_pred))


col_0            0     1
raintomorrow            
0             9130  3110
1              633  1669
                   pre       rec       spe        f1       geo       iba       sup

          0       0.94      0.75      0.73      0.83      0.74      0.54     12240
          1       0.35      0.73      0.75      0.47      0.74      0.54      2302

avg / total       0.84      0.74      0.73      0.77      0.74      0.54     14542



> * Le modèle SVM a des performances relativement bonnes pour prédire la classe "0" (pas de pluie demain), mais des performances plus faibles pour prédire la classe "1" (pluie demain).
> * Le modèle a correctement prédit la classe "0" pour la grande majorité des échantillons (9130 sur 12240, soit une précision de 0.94), mais il a eu plus de difficulté à prédire la classe "1" (seulement 1669 prédictions correctes sur 2302, soit une précision de 0.35).
> * Le score F1 global est de 0.77, ce qui indique une performance globale moyenne. 

In [23]:
svm = SVC(gamma='scale', class_weight='balanced')
svm.fit(X_train, y_train)                         

preds = svm.predict(X_test)

pd.crosstab(y_test, preds)

col_0,0,1
raintomorrow,,
0,9480,2760
1,639,1663


In [25]:
svm = SVC(probability=True, gamma='scale') # 'probability= True' est nécessaire pour retourner les probas
svm.fit(X_train, y_train)                        # mais ralentit l'entraînement

threshold = 0.4 # Tester avec 0.4, 0.6, ...

probs = svm.predict_proba(X_test)
pred_class = (probs[:,1]>=threshold).astype('int')

pd.crosstab(y_test, pred_class)

# Calcul et affichage du rapport de classification équilibré à l'aide de la fonction classification_report_imbalanced()
print(classification_report_imbalanced(y_test, y_pred))


                   pre       rec       spe        f1       geo       iba       sup

          0       0.94      0.75      0.73      0.83      0.74      0.54     12240
          1       0.35      0.73      0.75      0.47      0.74      0.54      2302

avg / total       0.84      0.74      0.73      0.77      0.74      0.54     14542



In [24]:
# Calcul et affichage du rapport de classification équilibré à l'aide de la fonction classification_report_imbalanced()
print(classification_report_imbalanced(y_test, y_pred))


                   pre       rec       spe        f1       geo       iba       sup

          0       0.94      0.75      0.73      0.83      0.74      0.54     12240
          1       0.35      0.73      0.75      0.47      0.74      0.54      2302

avg / total       0.84      0.74      0.73      0.77      0.74      0.54     14542



* Precision (précision): C'est la proportion d'observations prédites comme positives (1) qui sont réellement positives. Pour la classe 0, la précision est de 0,94, ce qui signifie que 94% des observations prédites comme positives (pas de pluie demain) étaient correctes, tandis que pour la classe 1, la précision est de 0,35, ce qui indique que seulement 35% des observations prédites comme positives (pluie demain) étaient correctes.

* Recall (rappel): C'est la proportion d'observations réellement positives qui ont été correctement identifiées par le modèle. Pour la classe 0, le rappel est de 0,75, ce qui signifie que le modèle a correctement identifié 75% des observations de pluie, tandis que pour la classe 1, le rappel est de 0,73, ce qui signifie que le modèle a correctement identifié 73% des observations de pas de pluie.

* Specificity (spécificité): C'est la proportion d'observations réellement négatives qui ont été correctement identifiées par le modèle. Pour la classe 0, la spécificité est de 0,73, ce qui signifie que le modèle a correctement identifié 73% des observations de pas de pluie, tandis que pour la classe 1, la spécificité est également de 0,75, ce qui signifie que le modèle a correctement identifié 75% des observations de pluie.

* F1-score: C'est une moyenne pondérée de la précision et du rappel, qui combine les deux métriques en une seule. Pour la classe 0, le score F1 est de 0,83, tandis que pour la classe 1, le score F1 est de 0,47.

* Geometric mean (moyenne géométrique): C'est la racine carrée du produit des taux de vrais positifs et de vrais négatifs, qui donne une mesure de la performance globale du modèle. Pour les deux classes, la moyenne géométrique est de 0,74.

* Balanced accuracy (exactitude équilibrée): C'est la moyenne arithmétique du rappel et de la spécificité, qui est une mesure de la précision globale du modèle pour les deux classes. Pour les deux classes, l'exactitude équilibrée est de 0,74.

Le rapport de classification équilibré montre que le modèle a une précision élevée pour la classe 0 (pas de pluie demain), mais une précision beaucoup plus faible pour la classe 1 (pluie demain). Le rappel est similaire pour les deux classes, mais la spécificité est légèrement plus élevée pour la classe 1. Le score F1 est plus élevé pour la classe 0 que pour la classe 1. La moyenne géométrique et l'exactitude équilibrée sont similaires pour les deux classes et indiquent une performance globale moyenne du modèle. Cela suggère que le modèle a plus de difficulté à prédire les jours avec pluie que les jours sans pluie.

#### <b>  Modification de la valeur de seuil de la classe 1 (pluie demain)

In [30]:
# Définition d'un seuil de décision pour la prédiction
threshold = 0.3 # Tester avec 0.4, 0.6, ...

# Prédiction des probabilités de chaque classe pour les données de test à l'aide du modèle SVM
probs = svm.predict_proba(X_test)

# Sélection de la classe prédite en fonction du seuil de décision
pred_class = (probs[:,1]>=threshold).astype('int')

# Affichage d'une matrice de confusion pour comparer les résultats de la prédiction avec les vraies étiquettes
print(pd.crosstab(y_test, pred_class))

# Affichage du rapport de classification qui fournit des mesures de performance pour chaque classe, y compris pour les classes déséquilibrées
print(classification_report_imbalanced(y_test, pred_class))


col_0             0     1
raintomorrow             
0             10969  1271
1               969  1333
                   pre       rec       spe        f1       geo       iba       sup

          0       0.92      0.90      0.58      0.91      0.72      0.54     12240
          1       0.51      0.58      0.90      0.54      0.72      0.50      2302

avg / total       0.85      0.85      0.63      0.85      0.72      0.53     14542



* Pour la classe 0, le modèle a une précision de 92%, ce qui signifie que lorsque le modèle prédit qu'il ne pleuvra pas, il a raison 92% du temps. Le rappel pour la classe 0 est de 90%, ce qui signifie que le modèle identifie correctement 90% des cas où il ne pleut pas. Le score F1 pour la classe 0 est de 91%, ce qui est une moyenne pondérée de la précision et du rappel. La spécificité pour la classe 0 est faible, à seulement 58%, ce qui signifie que le modèle a tendance à prédire des faux positifs (prédire qu'il va pleuvoir quand il ne pleut pas).

* Pour la classe 1, le modèle a une précision de 51%, ce qui signifie que lorsque le modèle prédit qu'il pleuvra, il a raison seulement 51% du temps. Le rappel pour la classe 1 est de 58%, ce qui signifie que le modèle identifie correctement seulement 58% des cas où il pleuvra. Le score F1 pour la classe 1 est de 54%, ce qui est une moyenne pondérée de la précision et du rappel. La spécificité pour la classe 1 est élevée, à 90%, ce qui signifie que le modèle a tendance à prédire des faux négatifs (prédire qu'il ne va pas pleuvoir quand il pleuvra).

* En général, les performances du modèle sont moyennes à bonnes pour la classe 0, mais médiocres pour la classe 1. Cela indique que le modèle a du mal à prédire les cas où il pleuvra, ce qui peut être problématique dans des situations où cette prédiction est cruciale. Les performances du modèle pourraient être améliorées en ajustant les paramètres du modèle, en utilisant des techniques de prétraitement de données ou en utilisant un algorithme de classification différent.

## 2ième modèle : BalancedRandomForestClassifier 


In [35]:
# Création d'une instance de la classe BalancedRandomForestClassifier
bclf = BalancedRandomForestClassifier()

# Entraînement du modèle sur les données d'entraînement
bclf.fit(X_train, y_train)

# Prédiction des étiquettes de classe pour les données de test en utilisant le modèle entraîné
y_pred = bclf.predict(X_test)

# Affichage de la table de contingence (matrice de confusion) pour évaluer la performance du modèle
print(pd.crosstab(y_test, y_pred))

# Importation de la fonction classification_report_imbalanced pour afficher le rapport de classification déséquilibré
from imblearn.metrics import classification_report_imbalanced

# Affichage du rapport de classification qui fournit des mesures de performance pour chaque classe, y compris pour les classes déséquilibrées
print(classification_report_imbalanced(y_test, y_pred))

col_0            0     1
raintomorrow            
0             9702  2538
1              423  1879
                   pre       rec       spe        f1       geo       iba       sup

          0       0.96      0.79      0.82      0.87      0.80      0.65     12240
          1       0.43      0.82      0.79      0.56      0.80      0.65      2302

avg / total       0.87      0.80      0.81      0.82      0.80      0.65     14542



* Dans cette analyse de résultats, nous avons utilisé le modèle BalancedRandomForestClassifier pour prédire les étiquettes de classe pour les données de test. 
La table de contingence (matrice de confusion) montre que le modèle a prédit correctement 9702 observations de la classe 0 (pas de pluie demain) et 1879 observations de la classe 1 (pluie demain), tandis que 423 observations de la classe 1 ont été mal classées en classe 0 et 2538 observations de la classe 0 ont été mal classées en classe 1.

* Le rapport de classification déséquilibré montre que le modèle a une précision élevée pour la classe 0 (96%) et une précision faible pour la classe 1 (43%). Cela signifie que le modèle est très précis pour prédire les observations de la classe 0, mais qu'il a tendance à faire plus d'erreurs pour prédire les observations de la classe 1. Le rappel est élevé pour la classe 1 (82%), ce qui indique que le modèle a identifié la plupart des observations de la classe 1, mais le rappel est faible pour la classe 0 (79%), ce qui indique que le modèle a manqué certaines observations de la classe 0. Le score F1 est de 0,56 pour la classe 1, ce qui signifie que le modèle a un compromis entre la précision et le rappel pour cette classe.

* En fin de compte, le modèle BalancedRandomForestClassifier semble mieux performant que le modèle SVM dans la prédiction de la classe déséquilibrée. Cependant, le modèle a encore besoin d'être amélioré pour être plus performant pour la classe 1.